In [1]:
# Внимание!!! Важно, что бы файлы с данными и исполняемый файл находились в одной папке, 
# тогда пути к тестовым и тренировочным наборам будут содержать только имена файлов.
# 
# В пути к тренировочным и тестовым данным запрежается использовать абсалютную адресацию, 
# то есть адресацию, в которой присутствуют имена папок. Путь должен содержать только имя файла.
#
# Напоминание: под моделью машинного обучения понимаются все действия с исходными данными, 
# которые необходимо произвести, что бы сопоставить признаки целевому значению.

### Область работы 1 (библиотеки)

In [2]:
# Данный блок в области 1 выполняется преподавателем
# 
# данный блок предназначен только для подключения необходимых библиотек
# запрещается подключать библиотеки в других блоках
# запрещается скрывать предупреждения системы
# установка дополнительных библиотек размещается прямо здесь (обязательно закоментированы)
# pip install

In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split 
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, Normalizer
from sklearn.metrics import roc_auc_score

from sklearn.metrics import r2_score, mean_squared_error, make_scorer
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

from sklearn.naive_bayes import BernoulliNB 

from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import roc_curve, roc_auc_score

from sklearn.model_selection import GridSearchCV, KFold, StratifiedShuffleSplit

from sklearn.model_selection import validation_curve
from sklearn.model_selection import ShuffleSplit

from sklearn.compose import ColumnTransformer, make_column_transformer, TransformedTargetRegressor

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer

from sklearn.preprocessing import OrdinalEncoder,LabelEncoder, OneHotEncoder
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures

import category_encoders as ce

from sklearn import svm
from sklearn.svm import SVC

import dtreeviz
from matplotlib.colors import ListedColormap
from sklearn.ensemble import RandomForestClassifier

from sklearn import tree
from sklearn.tree import export_graphviz
#import pydotplus
from IPython.display import Image

from sklearn.metrics import f1_score

from sklearn.ensemble import VotingClassifier

from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import cross_validate

from tqdm import tqdm

### Область работы 2 (выполнение лучшей модели)

In [4]:
# Данный блок(и) в области 2 выполняется преподавателем
#
# В области находится одна, единственная, итоговая модель машинного обучения с однозначными, 
# зафиксированными параметрами
#
# В данной области категорически запрещается искать, выбирать, улучшать, оптимизировать, 
# тюниговать и т.д. модель машинного обучения

In [2]:
# Путь к тренировочному набору
path_train = pd.read_csv('train.csv') # содержит только имя файла, без имен папок
# Путь к тестовому набору
path_test  = pd.read_csv('test.csv') # содержит только имя файла, без имен папок

In [6]:
# Блок(и) обучения и поверки модели

In [4]:
df = path_train 

,C1,N2,N3,C4_enc,C5_enc,C6_enc,N7,C8,C9,N10,C11,C12_enc,N13,N14,Target
0,1.0,28.00,2.000,2.0,4.0,8.0,4.165,1.0,1.0,2.0,1.0,2.0,181.0,1.0,1.0
1,1.0,36.25,5.000,2.0,8.0,5.0,2.500,1.0,1.0,6.0,0.0,2.0,0.0,368.0,1.0
2,0.0,29.50,2.000,1.0,10.0,8.0,2.000,0.0,0.0,0.0,0.0,2.0,256.0,18.0,0.0
3,1.0,22.67,0.165,2.0,8.0,3.0,2.250,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0
4,0.0,25.75,0.500,2.0,8.0,8.0,0.875,1.0,0.0,0.0,1.0,2.0,491.0,1.0,1.0


In [5]:
X = df.drop(['Target'], axis = 1)
y = df['Target']
cat_features = ['C4_enc','C5_enc','C6_enc','C12_enc']
axis_features = ['C1','C8','C9','C11']
num_features = ['N2', 'N3','N7','N10','N13','N14']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

In [7]:
axis_transformer = Pipeline(steps=[
    ('polynom', PolynomialFeatures(2,include_bias=False)),
    ('scaler', StandardScaler())])

CT = ColumnTransformer([
        ("pol_std", axis_transformer, axis_features),
        ("num", StandardScaler(), num_features),
        ("cat", OneHotEncoder(), cat_features)
        ])

display(CT)

res_ct = CT.fit_transform(X_train)
pd.DataFrame(res_ct).head()

ColumnTransformer(transformers=[('pol_std',
                                 Pipeline(steps=[('polynom',
                                                  PolynomialFeatures(include_bias=False)),
                                                 ('scaler', StandardScaler())]),
                                 ['C1', 'C8', 'C9', 'C11']),
                                ('num', StandardScaler(),
                                 ['N2', 'N3', 'N7', 'N10', 'N13', 'N14']),
                                ('cat', OneHotEncoder(),
                                 ['C4_enc', 'C5_enc', 'C6_enc', 'C12_enc'])])

,0,1,2,3,4,5,6,7,8,9,...,38,39,40,41,42,43,44,45,46,47
0,0.708401,0.952353,-0.889041,1.04491,0.708401,1.351882,-0.613057,1.396295,0.952353,-0.708401,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.708401,0.952353,1.124808,1.04491,0.708401,1.351882,1.631170,1.396295,0.952353,1.411631,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.708401,-1.050031,-0.889041,-0.95702,0.708401,-0.739709,-0.613057,-0.716181,-1.050031,-0.708401,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.708401,-1.050031,-0.889041,-0.95702,0.708401,-0.739709,-0.613057,-0.716181,-1.050031,-0.708401,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.708401,0.952353,1.124808,1.04491,0.708401,1.351882,1.631170,1.396295,0.952353,1.411631,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [8]:
column_trans = make_column_transformer(
    (KNNImputer(missing_values=np.nan, n_neighbors=3, weights="uniform"), ['N2', 'N3','N7','N10','N13','N14']) 
     )

X_ct = column_trans.fit_transform(X_train)

df_knn = pd.DataFrame(X_ct, columns = column_trans.get_feature_names_out(), index = X_train.index)
display(df_knn.head())
df_knn.describe()

,knnimputer__N2,knnimputer__N3,knnimputer__N7,knnimputer__N10,knnimputer__N13,knnimputer__N14
86,21.50,11.500,0.50,0.0,100.0,69.0
18,29.17,3.500,3.50,3.0,329.0,1.0
151,29.83,1.250,0.25,0.0,224.0,1.0
203,37.50,0.835,0.04,0.0,120.0,6.0
359,20.50,2.415,2.00,11.0,200.0,3001.0


,knnimputer__N2,knnimputer__N3,knnimputer__N7,knnimputer__N10,knnimputer__N13,knnimputer__N14
count,410.000000,410.000000,410.000000,410.000000,410.000000,410.000000
mean,32.139073,4.727622,2.432427,2.490244,173.073171,1157.648780
std,12.173578,4.733643,3.525042,4.362673,143.922179,6084.050311
min,13.750000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,22.940000,1.095000,0.210000,0.000000,60.500000,1.000000
50%,29.540000,3.000000,1.000000,0.000000,160.000000,7.000000
75%,38.730000,7.030000,3.085000,3.000000,260.000000,501.000000
max,80.250000,28.000000,28.500000,40.000000,720.000000,100001.000000


In [17]:
pipe = Pipeline([('preprocessing', [MinMaxScaler(), RobustScaler(),StandardScaler()]), 
                 ('regressor',    KNeighborsClassifier())])

kfold = KFold(n_splits=10, shuffle=True)


p = [2]
weights = ['uniform']
n_neighbors = [94]
scaling = [MinMaxScaler()]

param_grid =[
    {'preprocessing': scaling, 
     'regressor': [KNeighborsClassifier()],
         'regressor__n_neighbors': n_neighbors,
         'regressor__p': p,
         'regressor__weights': weights
     }]

grid = GridSearchCV(pipe, param_grid, cv=kfold, scoring='f1', n_jobs = -1,return_train_score=True)
grid.fit(X_train,y_train)
grid
print("Средняя правильность для наилучшей модели кроссвалидации на" '\n'
      "валидационных тестовых наборах: {:.6f}\n".format(grid.best_score_)) 
print("Правильность для наилучшей модели на тестовом наборе: {:.6f}\n".format(grid.score(X_test, y_test)))


Средняя правильность для наилучшей модели кроссвалидации на
валидационных тестовых наборах: 0.846746

Правильность для наилучшей модели на тестовом наборе: 0.876543



In [42]:
porogi = np.unique(grid.predict_proba(X_test)[:,1])
proba = grid.predict_proba(X_test)[:,1]

max_f1 = 0
max_por = 0

y_pred = grid.predict_proba(X_test)[:,1]

for i in porogi:
    y_pred_class = (y_pred>i).astype(int)
    f1_c = f1_score(y_test,y_pred_class)
    
    if f1_c > max_f1:
        max_f1 = f1_c
        max_por = i
print(max_f1, i)

0.8641975308641975 0.8936170212765957


In [46]:
f1_list = []
for i in tqdm(range(10)): 
    
    X_train, X_test, y_train, y_test = train_test_split(df.drop(columns = ['Target']), df.Target, test_size=0.3,
                                                    stratify=df.Target)   
    grid.fit(X_train, y_train)
    
    y_pred = grid.predict(X_test)
    
    f1_list.append(f1_score(y_test, y_pred))
    
mean_f1 = np.array(f1_list).mean()
std_f1 = np.array(f1_list).std()

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  5.68it/s]


In [47]:
print(f'CI(f1) = [{mean_f1 - 1.96*(std_f1)}; {mean_f1 + 1.96*(std_f1)}]')
print(f'mean f1: {mean_f1}')

CI(f1) = [0.8225841945807296; 0.9008051818703585]
mean f1: 0.8616946882255441


In [25]:
clf = make_pipeline(MinMaxScaler(), SVC(gamma='auto'))
clf.fit(X_train, y_train)
pipe = Pipeline([('preprocessing', [MinMaxScaler()]), 
                 ('classifier', svm.SVC())])

kfold = KFold(n_splits=10, shuffle=True)


C=[0.1]
gamma=[1]

param_grid =[
    {'preprocessing': [MinMaxScaler()], 
     'classifier': [svm.SVC()],
     'classifier__C': C,
     'classifier__gamma': gamma,
     'classifier__probability' : [True]
     }
]

grid1 = GridSearchCV(pipe, param_grid, cv=kfold, return_train_score=True, n_jobs = -1)
grid1.fit(X_train,y_train)

print("Средняя правильность для наилучшей модели кроссвалидации на" '\n'
      "валидационных тестовых наборах: {:.6f}\n".format(grid.best_score_)) 
print("Правильность для наилучшей модели на тестовом наборе: {:.6f}\n".format(grid1.score(X_test, y_test)))

Средняя правильность для наилучшей модели кроссвалидации на
валидационных тестовых наборах: 0.846746

Правильность для наилучшей модели на тестовом наборе: 0.857955



In [43]:
porogi = np.unique(grid1.predict_proba(X_test)[:,1])
proba = grid1.predict_proba(X_test)[:,1]

max_f1 = 0
max_por = 0

y_pred = grid1.predict_proba(X_test)[:,1]

for i in porogi:
    y_pred_class = (y_pred>i).astype(int)
    f1_c = f1_score(y_test,y_pred_class)
    
    if f1_c > max_f1:
        max_f1 = f1_c
        max_por = i
print(max_f1, i)

0.8674698795180724 0.933040452591211


In [48]:
f1_list = []
for i in tqdm(range(10)): 
    
    X_train, X_test, y_train, y_test = train_test_split(df.drop(columns = ['Target']), df.Target, test_size=0.3,
                                                    stratify=df.Target)   
    grid1.fit(X_train, y_train)
    
    y_pred = grid1.predict(X_test)
    
    f1_list.append(f1_score(y_test, y_pred))
    
mean_f1 = np.array(f1_list).mean()
std_f1 = np.array(f1_list).std()

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  5.30it/s]


In [49]:
print(f'CI(f1) = [{mean_f1 - 1.96*(std_f1)}; {mean_f1 + 1.96*(std_f1)}]')
print(f'mean f1: {mean_f1}')

CI(f1) = [0.7864891975454045; 0.8824799443295199]
mean f1: 0.8344845709374622


In [18]:
rf1 = RandomForestClassifier()

pipe = Pipeline([('preprocessing', [MinMaxScaler(), RobustScaler(), StandardScaler()]), 
                 ('classifier', rf1)])

kfold = KFold(n_splits=10, shuffle=True)

param_grid = [
    { 
     
    'n_estimators': [500],
    'max_features': ['log2'],
    'max_depth' : [8],
    'criterion' : ['gini'],
    'min_samples_leaf' : [1]
}
             ]
CV_rf = GridSearchCV(rf1, param_grid=param_grid, cv= kfold, n_jobs = -1, return_train_score=True)
CV_rf.fit(X_train, y_train)
print("Средняя правильность для наилучшей модели кроссвалидации на" '\n'
      "валидационных тестовых наборах: {:.6f}\n".format(grid.best_score_)) 
print("Правильность для наилучшей модели на тестовом наборе: {:.6f}\n".format(grid1.score(X_test, y_test)))

Средняя правильность для наилучшей модели кроссвалидации на
валидационных тестовых наборах: 0.846746

Правильность для наилучшей модели на тестовом наборе: 0.857955



In [44]:
porogi = np.unique(CV_rf.predict_proba(X_test)[:,1])
proba = CV_rf.predict_proba(X_test)[:,1]

max_f1 = 0
max_por = 0

y_pred = CV_rf.predict_proba(X_test)[:,1]

for i in porogi:
    y_pred_class = (y_pred>i).astype(int)
    f1_c = f1_score(y_test,y_pred_class)
    
    if f1_c > max_f1:
        max_f1 = f1_c
        max_por = i
print(max_f1, i)

0.967741935483871 0.9971042857142858


In [50]:
f1_list = []
for i in tqdm(range(10)): 
    
    X_train, X_test, y_train, y_test = train_test_split(df.drop(columns = ['Target']), df.Target, test_size=0.3,
                                                    stratify=df.Target)   
    CV_rf.fit(X_train, y_train)
    
    y_pred = CV_rf.predict(X_test)
    
    f1_list.append(f1_score(y_test, y_pred))
    
mean_f1 = np.array(f1_list).mean()
std_f1 = np.array(f1_list).std()

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:26<00:00,  2.65s/it]


In [51]:
print(f'CI(f1) = [{mean_f1 - 1.96*(std_f1)}; {mean_f1 + 1.96*(std_f1)}]')
print(f'mean f1: {mean_f1}')

CI(f1) = [0.835851970857911; 0.8915618569783955]
mean f1: 0.8637069139181532


In [19]:
clf_v1 = grid
clf_v2 = grid1
clf_v3 = CV_rf

In [53]:
e_s_clf = StackingClassifier(
        estimators = [
            ('knn', clf_v1), 
            ('svm', clf_v2), 
            ('rf', clf_v3)],
        final_estimator = RandomForestClassifier(n_estimators=50),
        stack_method = 'predict',
        n_jobs = -1
)

e_s_clf.fit(X_train,y_train)

print("Правильность для наилучшей модели на тестовом наборе: {:.6f}\n".format(e_s_clf.score(X_test, y_test)))

Правильность для наилучшей модели на тестовом наборе: 0.909091



In [55]:
f1_list = []
for i in tqdm(range(10)): 
    
    X_train, X_test, y_train, y_test = train_test_split(df.drop(columns = ['Target']), df.Target, test_size=0.3,
                                                    stratify=df.Target)   
    e_s_clf.fit(X_train, y_train)
    
    y_pred = e_s_clf.predict(X_test)
    
    f1_list.append(f1_score(y_test, y_pred))
    
mean_f1 = np.array(f1_list).mean()
std_f1 = np.array(f1_list).std()

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [07:27<00:00, 44.72s/it]


In [56]:
print(f'CI(f1) = [{mean_f1 - 1.96*(std_f1)}; {mean_f1 + 1.96*(std_f1)}]')
print(f'mean f1: {mean_f1}')

CI(f1) = [0.8001124659837832; 0.9034379859543521]
mean f1: 0.8517752259690676


Долго думала, в итоге боле доверяю метамодельке, несмотря на невыгодный матож. Прошу понять и простить!

Бустинга не будет, пока не разобралась. К контрольной пойму, приму и осознаю, честное слово!

In [7]:
# Блок предсказания с использованием тестового набора

In [40]:
y_predicted = e_s_clf.predict(X_test)

In [41]:
# Название вектора предсказанных значений  y_predict полученого на основании тестового набора
y_predict = y_predicted
y_predict

array([1., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0.,
       1., 1., 1., 0., 1., 1., 0., 1., 0., 0., 1., 1., 0., 0., 1., 1., 0.,
       0., 1., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 1., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 1., 0.,
       0., 0., 1., 0., 1., 1., 1., 1., 0., 0., 0., 0., 1., 1., 0., 0., 1.,
       0., 1., 0., 1., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1.,
       0., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 1., 1.,
       0., 1., 0., 0., 0., 0., 1., 1., 1., 0., 1., 0., 1., 1., 0., 1., 0.,
       1., 0., 1., 0., 1., 0., 1., 1., 0., 1., 0., 1., 1., 0., 0., 0., 0.,
       0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0.,
       1., 0., 0., 1., 0., 0.])